In [23]:
%pip install mesa

Note: you may need to restart the kernel to use updated packages.


In [24]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import time
import datetime


class CleaningRobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        (x, y) = self.pos
        if self.model.dirty_cells[x][y] == 1:
            self.model.dirty_cells[x][y] = 0

        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        vacant_cells = [cell for cell in neighborhood if self.model.grid.is_cell_empty(cell)]
        if vacant_cells:
            new_position = self.random.choice(vacant_cells)
            self.model.grid.move_agent(self, new_position)


class CleaningModel(Model):
    def __init__(self, width, height, num_vacuums, dirty_percentage):
        self.num_vacuums = num_vacuums
        self.total_cells = width * height
        self.amount_of_dirty_cells = int(self.total_cells * (dirty_percentage / 100))
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.dirty_cells = np.zeros((width, height))
        self.datacollector = DataCollector(
            model_reporters={"Dirty_Percentage": lambda m: (np.sum(m.dirty_cells) / self.total_cells) * 100},
            agent_reporters={"Position": "pos"}
        )

        for i in range(self.num_vacuums):
            agent = CleaningRobotAgent(i, self)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x, y))

        count = 0
        while count < self.amount_of_dirty_cells:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                count += 1

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


def run_simulation(width, height, num_vacuums, dirty_percentage, max_steps=None):
    print(f"Running simulation for {max_steps if max_steps else 'unlimited'} steps")
    start_time = time.time()
    model = CleaningModel(width, height, num_vacuums, dirty_percentage)
    steps = 0

    while max_steps is None or steps < max_steps:
        model.step()
        if np.sum(model.dirty_cells) == 0:
            break
        steps += 1

    dirty_percentage = (np.sum(model.dirty_cells) / model.total_cells) * 100
    print("Percentage of dirty cells:", dirty_percentage)
    print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))
    print(f'Number of steps: {steps}')


# Run simulations
run_simulation(100, 100, 1, 90, 100)
run_simulation(100, 100, 2, 90, 100)
run_simulation(100, 100, 1, 90, 1000)
run_simulation(100, 100, 2, 90, 1000)
run_simulation(100, 100, 1, 90, 10000)
run_simulation(100, 100, 2, 90, 10000)

Running simulation for 100 steps
Percentage of dirty cells: 89.49000000000001
Execution Time: 0:00:00.025166
Number of steps: 100
Running simulation for 100 steps
Percentage of dirty cells: 89.01
Execution Time: 0:00:00.022999
Number of steps: 100
Running simulation for 1000 steps
Percentage of dirty cells: 86.52
Execution Time: 0:00:00.134616
Number of steps: 1000
Running simulation for 1000 steps
Percentage of dirty cells: 83.81
Execution Time: 0:00:00.049637
Number of steps: 1000
Running simulation for 10000 steps
Percentage of dirty cells: 63.32
Execution Time: 0:00:00.213081
Number of steps: 10000
Running simulation for 10000 steps
Percentage of dirty cells: 51.23
Execution Time: 0:00:00.320373
Number of steps: 10000


In [25]:
run_simulation(100, 100, 1, 90)

Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:05.392225
Number of steps: 252349


In [18]:
run_simulation(100, 100, 2, 90)

Percentage of dirty cells: 0.0
Execution Time: 0:00:03.873664
Number of steps: 151065


In [19]:
run_simulation(100, 100, 3, 90)
run_simulation(100, 100, 4, 90)
run_simulation(100, 100, 5, 90)
run_simulation(100, 100, 6, 90)
run_simulation(100, 100, 7, 90)

Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:02.753752
Number of steps: 81142
Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:04.521366
Number of steps: 115849
Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:02.371378
Number of steps: 52899
Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:03.061572
Number of steps: 63555
Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:02.674607
Number of steps: 48376


In [20]:
run_simulation(100, 100, 8, 90)

Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:02.151025
Number of steps: 36383


In [21]:
run_simulation(100, 100, 9, 90)

Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:01.335900
Number of steps: 22137


In [22]:
run_simulation(100, 100, 10, 90)

Running simulation for unlimited steps
Percentage of dirty cells: 0.0
Execution Time: 0:00:01.785660
Number of steps: 25270
